In [76]:
#Imports libraries and loads data into pandas dataframes (Same as main program)

import numpy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.io.json import json_normalize
from pandas.tseries import converter as pdtc
from sklearn.svm import SVR
import matplotlib.units as munits
from keras.layers import Dropout
from keras.regularizers import l2
from datetime import datetime
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from sklearn.linear_model import LinearRegression
from scipy import stats
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from functools import reduce
from datetime import datetime
from keras.optimizers import SGD
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import read_csv
from pandas.plotting import scatter_matrix
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Lambda, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras
import json
import os
import sklearn
import numpy
from datetime import datetime
from dateutil.parser import parse
import findspark
findspark.init()
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
import pyspark
pd.set_option('display.max_rows', None)

#Initialize spark session
spark = SparkSession \
        .builder \
        .appName("ensemble_learning_sparktest") \
        .getOrCreate()

def concatenate_json_to_df(directory_path, to_output, folder_output):
    list_json = []
    
    for file in os.listdir(directory_path):
        if file.split(".")[-1] == "json":
            
            with open(directory_path+file, 'r') as f:
            
                # If json file is not empty
                if os.stat(directory_path+file).st_size != 0:
                                
                        
                    # For Observation file type
                    if directory_path.split("/")[-2] == "Observations":
                        dictio = json.load(f)
                                            
                        if 'observation' in dictio:
                            observation = dictio["observation"]
                        
                            for entry in observation:
                                entry['date'] =  file.rstrip(".json")
                                entry['station'] = dictio['station']
                                entry['time'] = datetime.utcfromtimestamp(entry['time']).strftime('%d-%m-%Y %H:%M:%S')
                                list_json.append(entry)
                    
                    
                    # For Analyses file type
                    elif directory_path.split("/")[-2] == "Analyses":
                        dictio = json.load(f)
                        if 'source' in dictio and 'temperature' in dictio:
                            source = list(dictio['source'])
                            temperature = list(dictio['temperature'])
                            
                            dictio['date'] = file.rstrip(".json")
                            dictio.pop('source', None)
                            dictio.pop('temperature', None)
                            
                            entry = dict()
                            for i in range(0, len(source)):
                                dictio[source[i]] = temperature[i]
                                list_json.append(dictio)

                                
                    # For Previsions file type
                    elif "Previsions" in directory_path.split("/"):
                        dictio = json.load(f)
                        
                        latit_r = None
                        long_r = None
                        
                        if "longitude_r" in dictio:
                            long_r = dictio['longitude_r']

                        if "latitude_r" in dictio:
                            latit_r = dictio['latitude_r']
                            
                        if 'forecast' in dictio:
                            forecast = dictio['forecast'] 
                            
                            
                            for i in range(0, len(forecast)):
                                dictio = dict()
                                
                                if long_r != None and latit_r != None:
                                    dictio['latitude_r'] = latit_r
                                    dictio['longitude_r'] = long_r
                                
                                if "temperature" in forecast[i]:
                                    dictio["temperature"] = forecast[i]["temperature"]
                                    
                                if "time" in forecast[i]:
                                    dictio["time"] = datetime.utcfromtimestamp(forecast[i]["time"]).strftime('%d-%m-%Y %H:%M:%S')
                                    
                                if "wind" in forecast[i]:
                                    dictio["wind"] = forecast[i]["wind"]
                                
                                if "wind_dir" in forecast[i]:
                                    dictio["wind_dir"] = forecast[i]["wind_dir"]
                                
                                if "humidity" in forecast[i]:
                                    dictio["humidity"] = forecast[i]["humidity"]
                                
                                if "pressure" in forecast[i]:
                                            dictio["pressure"] = forecast[i]["pressure"]
                                        
                                if "cloud_cover" in forecast[i]:
                                            dictio["cloud_cover"] = forecast[i]["cloud_cover"]

    
                                
                                dictio["date"] = file.rstrip(".json")
                                
                                list_json.append(dictio)
                
                
                else:
                    dictio = dict()
                    dictio["date"] = file.rstrip(".json")
                    list_json.append(dictio)
                                    
    df = pd.DataFrame(list_json)
    df.drop_duplicates()
    df.to_csv(folder_output+"/"+to_output.replace("/", "-")+".csv", sep=";", index=False)
    
    
dirname = "technical_challenge_v9"
if not os.path.exists(dirname):
    os.mkdir(dirname)
    
concatenate_json_to_df('technical_challenge/Asport CAPRED/Previsions/Api-Agro/', "Asport CAPRED/Previsions/Api-Agro", dirname)
# Launch data of Api, Dark and weatherbit forcesaters for the station Asport
path = "technical_challenge_v9/Asport CAPRED-Previsions-"
path_previsions = ["Api-Agro.csv", "Dark Sky.csv", "Weatherbit.csv"]
path_observations = "technical_challenge_v9/Asport CAPRED-Observations.csv"

def get_forecast(path, path_previsions, path_observations):
    list_data = []
    for forecaster in path_previsions:
        dataframe = pd.read_csv(path + forecaster, sep=";")
        dataframe["time"] = pd.to_datetime(dataframe['time'], format='%d-%m-%Y %H:%M:%S')
        dataframe.sort_values(by=['time'])
        dataframe['time'] = dataframe['time'].apply(lambda x: x.replace(minute=0, second=0))
        dataframe = dataframe.groupby(['time']).apply(lambda x : x.iloc[:6].mean())
        dataframe.columns = [x + "_" + forecaster.split('.')[0] for x in dataframe.columns]
        list_data.append(dataframe)   
    
    observation_data = pd.read_csv(path_observations, sep=";", index_col=False).drop(columns=["station"]).dropna()
    observation_data["time"] = pd.to_datetime(observation_data['time'], format='%d-%m-%Y %H:%M:%S')
    observation_data.sort_values(by='time')
    observation_data['time'] = observation_data['time'].apply(lambda x: x.replace(minute=0, second=0))
    observation_data = observation_data.groupby(['time']).apply(lambda x : x.iloc[0])
    observation_data=observation_data.rename(index=str, columns={"humidity": "humidity_Y", "precipitation": "precipitation_Y", "temperature":"temperature_Y"})
    
    return list_data, observation_data

# Associate X to previsions and Y to observations 
list_data, observation_data = get_forecast(path, path_previsions, path_observations)
X = pd.concat(list_data, axis=1)
Y = observation_data

# Delete non useful variable in prevision data
X = X.loc[:, ~X.columns.str.startswith('l')]
X = X.loc[:, ~X.columns.str.startswith('c')]
X = X.loc[:, ~X.columns.str.startswith('p')]
X = X.loc[:, ~X.columns.str.startswith('wind_dir')]
X = X.loc[:, ~X.columns.str.startswith('prec')]
X = X.loc[:, ~X.columns.str.startswith('date')]
X = X.loc[:, ~X.columns.str.startswith('time')]

# Delete non useful variable in observation data
Y = Y.drop(['date','time'], axis=1)

# Import JSON API in order to collect most recent data
import json,urllib.request
data_A = urllib.request.urlopen("http://sd-59247.dedibox.fr/agriscope/obs/transform_obs.php?STATION=Asport").read()
output_A = json.loads(data_A)

# Transform JSON data in a dataframe A
A = json.loads(data_A)
A_data = pd.DataFrame(A)
A = A_data.T

# Delelte the index of the dataframe A
A.index.name = 'time'
A.reset_index(level=0, inplace=True)

# Convert date time in correct format 
A['time'] = pd.to_datetime(A['time'])
A['time'] = pd.to_datetime(A['time'],'%d-%m-%Y %H:%M:%S')
A['time'] = A['time'].apply(lambda x: x.replace(minute=0, second=0))

# Rename dataframe A columns
A.columns = ['time', 'humidity_Y', 'precipitation_Y', 'temperature_Y']

# Reset the index of the dataframe A
A = A.set_index('time') 

# Concat vertically Y and A (Y contains former observation and A most recent one)
frames = [Y, A]
first_concat = pd.concat(frames)

# Merge X and Y dataframe in one single dataframe "df_forcast_cap"
df_forcast_cap = pd.merge(X, first_concat, right_index=True, left_index=True)
df_forcast_cap = df_forcast_cap.drop(['precipitation_Y'], axis=1)

# Delete missing values for "df_forcast_cap"
df_forcast_cap = df_forcast_cap.dropna()

In [77]:
# Pyarrow optimization for pandas/spark conversion 

#spark.conf.set("spark.sql.execution.arrow.enabled", "true") NEED TO FIX PyARROW VERSION

In [78]:
#convert pandas to dataframe
train_df = spark.createDataFrame(df_forcast_cap) #dataframe

print(train_df)

/Users/will/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/sql/session.py:753: UserWarning:

createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.13.0.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.



DataFrame[temperature_Api-Agro: double, wind_Api-Agro: double, humidity_Api-Agro: double, humidity_Dark Sky: double, temperature_Dark Sky: double, wind_Dark Sky: double, humidity_Weatherbit: double, temperature_Weatherbit: double, wind_Weatherbit: double, humidity_Y: double, temperature_Y: double]


In [66]:
#Compress data points into feature vector
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["temperature_Api-Agro", "wind_Api-Agro", "humidity_Api-Agro", \
                                      "humidity_Dark Sky", "temperature_Dark Sky", "wind_Dark Sky", \
                                      "humidity_Weatherbit", "temperature_Weatherbit", "wind_Weatherbit", \
                                      "humidity_Y", "temperature_Y"],outputCol="features")
newdf = assembler.transform(train_df)

newdf.printSchema()

root
 |-- temperature_Api-Agro: double (nullable = true)
 |-- wind_Api-Agro: double (nullable = true)
 |-- humidity_Api-Agro: double (nullable = true)
 |-- humidity_Dark Sky: double (nullable = true)
 |-- temperature_Dark Sky: double (nullable = true)
 |-- wind_Dark Sky: double (nullable = true)
 |-- humidity_Weatherbit: double (nullable = true)
 |-- temperature_Weatherbit: double (nullable = true)
 |-- wind_Weatherbit: double (nullable = true)
 |-- humidity_Y: double (nullable = true)
 |-- temperature_Y: double (nullable = true)
 |-- features: vector (nullable = true)



In [67]:
#Creates df with temperature_Y and feature vector
newdf = newdf.selectExpr("temperature_Y as label", "features as features")

newdf.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [73]:
# Split the data into training and test sets
(trainingData, testData) = newdf.randomSplit([0.7, 0.3])   

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")
model = rf.fit(newdf)

In [74]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Compute RMS error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
    
spark.stop()

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 33.70148060517556| 30.0|[13.6412908333333...|
|35.672619179325146| 32.0|[12.6293786666666...|
| 34.26207057076552| 33.0|[14.70763075,16.9...|
|  36.7622389664255| 37.0|[14.3207596666666...|
| 39.43285112812153| 38.0|[18.2214705,11.00...|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2.28889


In [85]:
# GBR 

# Initialize GBT
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol="features", labelCol = "label", maxIter=10)

# Train model using same data structure as RandomForest
model = gbt.fit(newdf)

Py4JJavaError: An error occurred while calling o1733.fit.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2108)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2137)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2156)
	at org.apache.spark.rdd.RDD.$anonfun$take$1(RDD.scala:1423)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.RDD.take(RDD.scala:1396)
	at org.apache.spark.ml.tree.impl.DecisionTreeMetadata$.buildMetadata(DecisionTreeMetadata.scala:119)
	at org.apache.spark.ml.tree.impl.RandomForest$.run(RandomForest.scala:126)
	at org.apache.spark.ml.regression.DecisionTreeRegressor.$anonfun$train$2(DecisionTreeRegressor.scala:143)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrum

In [ ]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)